### Import Dependencies

In [1]:
import random
random.seed(0)

In [37]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.transforms import v2

### Constant Variables

In [3]:
DATASET_PATH = './knee-osteoarthritis_2'

In [4]:
TRAIN_PATH = f'{DATASET_PATH}/train'
VAL_PATH = f'{DATASET_PATH}/val'
TEST_PATH = f'{DATASET_PATH}/test'

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


### Dataset 

In [6]:
from src.dataset.augmented_dataset import get_KneeOsteoarthritis_Edges, KneeOsteoarthritis_Edges

# train_dataset = get_KneeOsteoarthritis_Edges(TRAIN_PATH)
# val_dataset = get_KneeOsteoarthritis_Edges(VAL_PATH)
transform_toTensor = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.ImageFolder(TRAIN_PATH, transform_toTensor)
# train_size = len(train_set)
val_dataset = torchvision.datasets.ImageFolder(VAL_PATH, transform_toTensor)
# val_size = len(val_set)
test_dataset = torchvision.datasets.ImageFolder(TEST_PATH, transform_toTensor)
# test_size = len(test_set)

dataset_all = torch.utils.data.ConcatDataset([train_dataset, val_dataset, test_dataset])

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset_all, [0.7, 0.1, 0.2])
train_dataset = KneeOsteoarthritis_Edges(train_dataset)
val_dataset = KneeOsteoarthritis_Edges(val_dataset)
test_dataset = KneeOsteoarthritis_Edges(test_dataset)

In [7]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

4736 676 1353


In [8]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

In [9]:
row = train_dataset[1]
normal_ex = row[0]
augmented_ex = row[1]
print(normal_ex.shape, augmented_ex.shape)

torch.Size([3, 256, 256]) torch.Size([1, 224, 224])


### Data Loader

In [10]:
from torch.utils.data import DataLoader
from src.other import getWeightedDataLoader
BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# val_loader = getWeightedDataLoader(val_dataset, BATCH_SIZE)

### Model

In [11]:
class IntermediarySpaceModel(nn.Module):
    def __init__(self, num_classes: int = 5, dropout: float = 0.5) -> None:
        super().__init__()
        
        # Size of layer block
        S = 24
        
        # Images
        self.imagesClassifier = nn.Sequential(
            nn.Conv2d(3, S*2, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=dropout*0.2),
            nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=dropout*0.4),
            nn.Conv2d(S*2, S*2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout*0.6),
            nn.Conv2d(S*2, S, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Flatten(),
            nn.Dropout(p=dropout*0.8),
            nn.Linear(S * 7 * 7, S*2),
        )

        self.edgesClassifier = nn.Sequential(
            nn.Conv2d(1, S*2, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=dropout*0.4),
            nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=dropout*0.6),
            nn.Conv2d(S*2, S, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Flatten(),
            nn.Dropout(p=dropout*0.8),
            nn.Linear(S * 6 * 6, S*2),
        )
        
        self.outputCombiner = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(S*4, S*3),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(S*3, S),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(S, num_classes),
        )

    def forward(self, images: torch.Tensor, edges: torch.Tensor) -> torch.Tensor:
        
        # Images
        images = self.imagesClassifier(images)
        
        # Edges
        edges = self.edgesClassifier(edges)
        
        # Combining outputs
        concated = torch.cat((images, edges), 1)
        res = self.outputCombiner(concated)
        
        return res

In [12]:
# from src.models.custom import CustomModel

# model = EarlyIntermediarySpaceModel(3, 0)
model = IntermediarySpaceModel(4, 0.4)
model = model.to(device)

In [13]:
# print(sum(p.numel() for p in net.classifier.parameters()) ,sum(p.numel() for p in net.edgesClassifier.parameters()) )
print(sum(p.numel() for p in model.parameters()))

trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())
print(sum(p.numel() for p in trainable_parameters))

287044
287044


### Training Loop

#### Setting optimizer

In [45]:
import torch.optim as optim
from src.other import getClassesFrequency

class_weights = getClassesFrequency(train_dataset)
weights_tensor = torch.Tensor(list(class_weights.values())).to(device)
print(class_weights, weights_tensor)

criterion = nn.CrossEntropyLoss(weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

{0: 2293, 2: 752, 3: 167, 1: 1524} tensor([2293.,  752.,  167., 1524.], device='cuda:0')


In [46]:
decayRate100 = 0.5
decayRate1 = decayRate100**(1/100)
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate1)

print(decayRate1)

0.9930924954370359


In [47]:
def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']

#### Setting Logger

In [48]:
# EXP_NAME = "5/0"
# from torch.utils.tensorboard import SummaryWriter

# logger = SummaryWriter(log_dir=f"logs/experiments/{EXP_NAME}")

In [49]:
epochCounter = 0

#### Training Loop

In [50]:
from src.validation import validate

def train_many(model, epochs_nr, logger = None, lr_scheduler = None, regularization_type = "L2", lambda_reg=0.01):
    global epochCounter
    
    for epoch in range(0, epochs_nr):  # loop over the dataset multiple times
        epoch_correct = 0
        epoch_samples = 0
        epoch_batches = 0
        running_loss = 0.0
    
        for i, data in enumerate(train_loader, 0):

            # get the inputs; data is a list of [inputs, labels]
            images, edges, labels = data
            images = images.to(device)
            edges = edges.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(images, edges)
            loss = criterion(outputs, labels)
            
            # Apply L1 regularization
            if regularization_type == 'L1':
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss += lambda_reg * l1_norm
                
            # Apply L2 regularization
            elif regularization_type == 'L2':
                l2_norm = sum(p.pow(2).sum() for p in model.parameters())
                loss += lambda_reg * l2_norm
                
            loss.backward()
            
            optimizer.step()
            
            # Changing outputs (logits) to labels
            outputs_clear = outputs.max(1).indices
            
            epoch_correct += (outputs_clear == labels).float().sum()
            epoch_samples += len(outputs)
            epoch_batches +=1
            
            running_loss += loss.item()
        
        tAccuracy = epoch_correct / epoch_samples * 100
        tLoss = running_loss / epoch_batches
        
        # Validation
        vAccuracy, vLoss = validate(model, val_loader, criterion, device)
        
        if logger != None:

            logger.add_text("REGULARIZATION_TYPE", regularization_type, global_step=epochCounter)
            logger.add_scalar("REGULARIZATION_LAMBDA", lambda_reg, global_step=epochCounter)
            logger.add_scalar("learning_rate", get_lr(optimizer), global_step=epochCounter)
            
            logger.add_scalar("Accuracy/train", tAccuracy, global_step=epochCounter)
            logger.add_scalar("Loss/train", tLoss, global_step=epochCounter)
            logger.add_scalar("Accuracy/validation", vAccuracy, global_step=epochCounter)
            logger.add_scalar("Loss/validation", vLoss, global_step=epochCounter)
        
        print(f'Epoch {epochCounter}: Training: accuracy: {tAccuracy:.3f}%, loss: {tLoss:.3f}; Validation: accuracy: {vAccuracy:.3f}%, loss: {vLoss:.3f}')
        
        epochCounter += 1
        
        if lr_scheduler != None:
            lr_scheduler.step()
        
        # print("lr= " + str(get_lr(optimizer)))
    print('Finished Training')

### Training Model

In [51]:
train_many(model, 60, None, my_lr_scheduler, "L2", 0.002)
# train_many(model, 50)

Epoch 0: Training: accuracy: 49.092%, loss: 0.643; Validation: accuracy: 47.485%, loss: 0.633
lr= 0.000993092495437036
Epoch 1: Training: accuracy: 50.612%, loss: 0.607; Validation: accuracy: 47.633%, loss: 0.547
lr= 0.0009862327044933591
Epoch 2: Training: accuracy: 51.499%, loss: 0.578; Validation: accuracy: 50.592%, loss: 0.565
lr= 0.0009794202975869268
Epoch 3: Training: accuracy: 54.688%, loss: 0.571; Validation: accuracy: 52.219%, loss: 0.583
lr= 0.0009726549474122855
Epoch 4: Training: accuracy: 55.194%, loss: 0.562; Validation: accuracy: 51.331%, loss: 0.620
lr= 0.0009659363289248456
Epoch 5: Training: accuracy: 55.954%, loss: 0.542; Validation: accuracy: 55.769%, loss: 0.569
lr= 0.0009592641193252644
Epoch 6: Training: accuracy: 57.179%, loss: 0.538; Validation: accuracy: 53.402%, loss: 0.580
lr= 0.0009526379980439374
Epoch 7: Training: accuracy: 58.678%, loss: 0.521; Validation: accuracy: 52.959%, loss: 0.537
lr= 0.000946057646725596
Epoch 8: Training: accuracy: 59.649%, loss

KeyboardInterrupt: 

In [ ]:
acc, loss = validate(model, test_loader, criterion, device)

print(acc, loss)

## Data Visualization

In [103]:
import sklearn
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm

In [130]:
def visualize_cm(cm, true_labels, predicted_labels):
    cmplot = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(4))
    cmplot.plot(cmap = 'Blues')
    plt.show()
    
def visualize(model, loader):
    correct = 0
    total = 0
    model.eval()
    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, edges, labels in tqdm(loader):
            inputs, edges, labels = inputs.to(device), edges.to(device), labels.to(device)
            outputs = model(inputs, edges)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

    print(classification_report(true_labels, predicted_labels, zero_division=np.nan))

    cm = confusion_matrix(true_labels, predicted_labels)
    visualize_cm(cm, true_labels, predicted_labels)

In [ ]:
visualize(model, train_loader)